In [346]:
# imports
import requests
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv

In [347]:
df_stations = pd.read_csv('df_stations.csv')[['id','name','total_bikes','latitude','longitude']]
df_stations.head()

,id,name,total_bikes,latitude,longitude
0,d334f65221f6a681f8b03433f36e2075,T19 Kiwanis Park,9,33.366488,-111.935612
1,cac663c57dee1dafd5f152bb906e97d5,P211 10th St / Pierce,6,33.455654,-112.060592
2,8d29ea89964d1eb73171310f718fe7e5,P196 2nd St. and Roosevelt,9,33.458995,-112.071128
3,6628d10945695a8a19a0ede32f949bc4,T17 Tempe Beach Park,23,33.429720,-111.940602
4,b953cdb6c1721e12c870e32cb780998c,ASU Goldwater Ctr. for Science - Tyler Mall (V...,6,33.420541,-111.930670


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [348]:
# Using dotenv to retrieve Foursquare API key
load_dotenv()
api_key_fs_string = 'FOURSQUARE_API_KEY' # replace with your Foursquare API key variable name
api_key_fs = os.getenv(api_key_fs_string)

In [384]:
# Function to find all POI's of a specific category within a set distance from any bike stations
def find_POIs_foursquare(station_ids,latitudes,longitudes,radius,limit,category_ids):
    
    fields = 'fsq_id,name,rating,popularity,price'
    list_POI = []
    
    # Find all POI's within 1000m of each bike station.  This will return one column with station_id and one column containing lists of POI's.
    for stat_ids,lat,long in zip(station_ids,latitudes,longitudes):
        url = f'https://api.foursquare.com/v3/places/search?ll={lat},{long}&radius={radius}&categories={category_ids}&limit={limit}&fields={fields}'
        headers = {"accept": "application/json","Authorization": api_key_fs}
        response = requests.get(url, headers=headers).json()['results']
        list_POI.append([stat_ids,response])
   
    df = pd.DataFrame(list_POI,columns=['station_id','results']) # convert to dataframe
    df = df.explode('results').reset_index(drop=True) # create 1 row for each match of bike station and POI.  There are duplicates of both bike stations and POI's, but each matchup is unique.
    df = pd.concat([df,pd.json_normalize(df['results'])],axis=1).drop('results',axis=1)
    return df

In [403]:
# Category IDs: Bars and Restaurants, 13003 and 13065.  Landmarks and Outdoors, 16000

# Creating dataframe for bars/restaurants
df_retaurants_bars_fs = find_POIs_foursquare(station_ids=df_stations['id'],
                                          latitudes=df_stations['latitude'],
                                          longitudes=df_stations['longitude'],
                                          radius=500,
                                          limit=50,
                                          category_ids='13003,13065')


df_retaurants_bars_fs.head()

,station_id,fsq_id,name,popularity,price,rating
0,d334f65221f6a681f8b03433f36e2075,NaN,NaN,NaN,NaN,NaN
1,cac663c57dee1dafd5f152bb906e97d5,5affb890efa82a002ca277b6,Welcome Diner,0.972645,1.0,8.6
2,cac663c57dee1dafd5f152bb906e97d5,59403814dec1d64454097bd6,Gallo Blanco,0.975962,1.0,8.5
3,cac663c57dee1dafd5f152bb906e97d5,57d77982498e47efcfc81c65,Cheba Hut,0.988500,1.0,9.0
4,cac663c57dee1dafd5f152bb906e97d5,5fa1d6942789095b87e8f481,Tacos Calafia - Downtown,0.987377,1.0,NaN


In [402]:
# Creating dataframe for Landmarks and Outdoors
df_landmarksOutdoors_fs = find_POIs_foursquare(station_ids=df_stations['id'],
                                                latitudes=df_stations['latitude'],
                                                longitudes=df_stations['longitude'],
                                                radius=1000,
                                                limit=50,
                                                category_ids='16000')
df_landmarksOutdoors_fs.head()

,station_id,fsq_id,name,popularity,rating
0,d334f65221f6a681f8b03433f36e2075,4ae46bedf964a5207e9a21e3,Kiwanis Park,0.999143,8.9
1,d334f65221f6a681f8b03433f36e2075,4e54654888770c1e522a95af,Kiwanis Park Soccer Fields,0.965342,NaN
2,d334f65221f6a681f8b03433f36e2075,4ea2ec928231f6dc6a3aa2b1,Tempe Canal,0.878511,NaN
3,d334f65221f6a681f8b03433f36e2075,4c047e1573a8c9b6993896e0,Kiwanis Volleyball Courts,0.898369,NaN
4,d334f65221f6a681f8b03433f36e2075,5278389011d2ec15155a70dc,La Estancia Doggie Run,0.104432,NaN


In [406]:
# Writing FourSquare dataframes to csv
df_landmarksOutdoors_fs.to_csv(path_or_buf='df_landmarksOutdoors_fs.csv',index=False)
df_retaurants_bars_fs.to_csv(path_or_buf='df_retaurants_bars_fs.csv',index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [352]:
# Using dotenv to retrieve Yelp API key
load_dotenv()
api_key_y_string = 'YELP_API_KEY2' # replace with your Yelp API key variable name
api_key_y = os.getenv(api_key_y_string)

In [399]:
# Function to find all POI's of a specific category within a set distance from any bike stations
def find_POIs_yelp(station_ids,latitudes,longitudes,radius,limit,categories):
    
    #fields = 'fsq_id,name,rating,popularity,price'
    list_POI = []
    
    # Find all POI's within 1000m of each bike station.  This will return one column with station_id and one column containing lists of POI's.
    for stat_ids,lat,long in zip(station_ids,latitudes,longitudes):
        url = f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius={radius}&limit={limit}&categories={categories}'
        headers = {"accept": "application/json","Authorization": 'bearer ' + api_key_y}
        response = requests.get(url, headers=headers).json()['businesses']
        list_POI.append([stat_ids,response])
   
    df = pd.DataFrame(list_POI,columns=['station_id','businesses']) # convert to dataframe
    df = df.explode('businesses').reset_index(drop=True) # create 1 row for each match of bike station and POI.  There are duplicates of both bike stations and POI's, but each matchup is unique.
    df = pd.concat([df,pd.json_normalize(df['businesses'])],axis=1).drop('businesses',axis=1)
    return df

In [396]:
# Using function to find Yelp POI's 
df_bars_yelp = find_POIs_yelp(station_ids=df_stations['id'],
                              latitudes=df_stations['latitude'],
                              longitudes=df_stations['longitude'],
                              radius=1000,
                              limit=50,
                              categories='bars')

df_landmarks_yelp = find_POIs_yelp(station_ids=df_stations['id'],
                                   latitudes=df_stations['latitude'],
                                   longitudes=df_stations['longitude'],
                                   radius=1000,
                                   limit=50,
                                   categories='landmarks')


In [405]:
# Writing Yelp dataframes to csv
df_bars_yelp.to_csv(path_or_buf='df_bars_yelp.csv',index=False)
df_landmarks_yelp.to_csv(path_or_buf='df_landmarks_yelp.csv',index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [ ]:
# Foursquare provided more numerical data, and Yelp orovided more categorical data

Get the top 10 restaurants according to their rating

In [408]:
# Foursquare top 10
df_retaurants_bars_fs.sort_values('rating',ascending=False).drop_duplicates('name').head(10)

,station_id,fsq_id,name,popularity,price,rating
1482,a702bd5692851de367b7b948ef6696d2,55600416498e5eb4dbecf791,Valley Bar,0.977446,2.0,9.2
838,ebb4ed2cfc9d563f8f15f9861a3ebccf,4a5fcb9cf964a5203cc01fe3,Postino Central,0.981354,2.0,9.2
488,03712bfd188583696010813718ea01e1,5085c381e4b038e4e19d739e,Federal Pizza,0.983153,2.0,9.2
1483,a702bd5692851de367b7b948ef6696d2,4e3a4cdf1850f0453401b5fd,Crescent Ballroom,0.992263,1.0,9.1
514,9be3ff8f22f184a0781c655b409bfbe6,4a3fdf4cf964a52004a41fe3,Cibo,0.977623,2.0,9.1
442,c1b890c63cf0a734dca62f21afc42b5d,502ac807e4b02d4e93003f14,Angels Trumpet Ale House,0.976980,2.0,9.1
184,ccfd57f453bbdacff8a335974083d103,52ce0ca6498e4e9046c318f9,El Chullo,0.964461,2.0,9.1
102,deb09562cebdde49d1c7d00a324464d9,57d77982498e47efcfc81c65,Cheba Hut,0.988500,1.0,9.0
546,57bfc33b0cd1895b52091365b125362c,472edc25f964a5200a4c1fe3,Matt's Big Breakfast,0.976993,2.0,9.0
672,3773b8b5382c51078c46ff144a9abad4,44e0acc6f964a52008371fe3,Cornish Pasty Co.,0.982057,2.0,9.0


In [410]:
# Yelp top 10
df_bars_yelp.sort_values('rating',ascending=False).drop_duplicates('name').head(10)

,station_id,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
1395,901440f4daab43ac6319ecbac2cada7f,H2HeOmDaztHe3TjLTj4YFw,food-on-wood-phoenix,Food On Wood,https://s3-media1.fl.yelpcdn.com/bphoto/n8JR6U...,False,https://www.yelp.com/biz/food-on-wood-phoenix?...,5.0,"[{'alias': 'wine_bars', 'title': 'Wine Bars'}]",5.0,...,33.466510,-112.074920,1616 N Central Ave,,None,Phoenix,85003,US,AZ,"[1616 N Central Ave, Phoenix, AZ 85003]"
1768,567004b9d63f86f35710d79bdb30c02e,WZS3WagNuA-APAX1Eqp26g,garden-bar-phoenix-phoenix,Garden Bar Phoenix,https://s3-media2.fl.yelpcdn.com/bphoto/rTObyi...,False,https://www.yelp.com/biz/garden-bar-phoenix-ph...,50.0,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,...,33.457650,-112.081641,822 N 6th Ave,,None,Phoenix,85003,US,AZ,"[822 N 6th Ave, Phoenix, AZ 85003]"
1796,567004b9d63f86f35710d79bdb30c02e,097Q9N5XyIDFDK0kQcyRjQ,moiselle-phoenix,Moiselle,https://s3-media1.fl.yelpcdn.com/bphoto/7snOoh...,False,https://www.yelp.com/biz/moiselle-phoenix?adju...,3.0,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",5.0,...,33.460829,-112.070796,1121 N 2nd St,None,,Phoenix,85004,US,AZ,"[1121 N 2nd St, Phoenix, AZ 85004]"
1023,a729c7bc4d432d61c6ea42c2a20e197a,95MO0h7grooyjn0lsJaNnw,ac-lounge-tempe,AC Lounge,,False,https://www.yelp.com/biz/ac-lounge-tempe?adjus...,3.0,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",5.0,...,33.430990,-111.937250,100 E Rio Salado Pkwy,,None,Tempe,85281,US,AZ,"[100 E Rio Salado Pkwy, Tempe, AZ 85281]"
1020,a729c7bc4d432d61c6ea42c2a20e197a,RD4DT9opiW_00Dq7FYaNng,university-sports-grill-tempe,University Sports Grill,https://s3-media2.fl.yelpcdn.com/bphoto/rypE2e...,False,https://www.yelp.com/biz/university-sports-gri...,5.0,"[{'alias': 'sportsbars', 'title': 'Sports Bars'}]",5.0,...,33.422698,-111.940372,740 S Mill Ave,Unit D 130,None,Tempe,85281,US,AZ,"[740 S Mill Ave, Unit D 130, Tempe, AZ 85281]"
976,ce9511579a675c959184e01c7bbee502,BfcjudgaW1dGgCx6eQz1zA,the-wayward-taphouse-phoenix,The Wayward Taphouse,https://s3-media4.fl.yelpcdn.com/bphoto/Eujc9A...,False,https://www.yelp.com/biz/the-wayward-taphouse-...,28.0,"[{'alias': 'beerbar', 'title': 'Beer Bar'}]",5.0,...,33.455252,-112.086770,1028 Grand Ave,,None,Phoenix,85007,US,AZ,"[1028 Grand Ave, Phoenix, AZ 85007]"
2486,cb4373b3cc89e041a72730400204f509,cYCXD-pCivbaEXwGwZICSQ,quartz-phoenix,Quartz,https://s3-media3.fl.yelpcdn.com/bphoto/FG58o9...,False,https://www.yelp.com/biz/quartz-phoenix?adjust...,29.0,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,...,33.451112,-112.078665,341 W Van Buren St,Ste B,,Phoenix,85003,US,AZ,"[341 W Van Buren St, Ste B, Phoenix, AZ 85003]"
308,80a8e77e11d0ff463c25a24317f99c62,TJI2YZ9oTcmh6CZE8uAzzA,el-chullo-peruvian-restaurant-phoenix,El Chullo Peruvian Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/NNT2DH...,False,https://www.yelp.com/biz/el-chullo-peruvian-re...,1.0,"[{'alias': 'peruvian', 'title': 'Peruvian'}, {...",5.0,...,33.500759,-112.082760,4414 N 7th Ave,,None,Phoenix,85013,US,AZ,"[4414 N 7th Ave, Phoenix, AZ 85013]"
1565,d442d37ff6ef724e22dad67d3ca21925,A4daIOZASO-wxDJzWhHD0g,van-daddies-phoenix,Van Daddies,,False,https://www.yelp.com/biz/van-daddies-phoenix?a...,2.0,"[{'alias': 'beerbar', 'title': 'Beer Bar'}]",5.0,...,33.444701,-112.069008,310 S 4th St,None,,Phoenix,85004,US,AZ,"[310 S 4th St, Phoenix, AZ 85004]"
2617,0cab138d1a9b55f141311a4a90e96d28,9CZ1u9xailwxWwgKbubgeA,concierge-bistro-bar-phoenix-2,Concierge Bistro Bar,https://s3-media2.fl.yelpcdn.com/bphoto/mPBkwk...,False,https://www.yelp.com/biz/concierge-bistro-bar-...,15.0,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",5.0,...,33.448470,-112.057240,1140 E Washington St,,None,Phoenix,85034,US,AZ,"[1140 E Washington St, Phoenix, AZ 85034]"
